<a href="https://colab.research.google.com/github/Tiabet/DACON_WebClick/blob/main/WebClick_LGBM_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import random
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_parquet('/content/drive/MyDrive/data/train_100000.parquet')
test = pd.read_parquet('/content/drive/MyDrive/data/test_100000.parquet')

In [3]:
train['F01'].value_counts()

F01
JCDXFYU    15636634
LLKAVMO     6532633
VNOHLIR     1455676
XOZOJFY     1062708
VFTOLQT      697032
TVWQYKH      483327
GLPZFND      464013
UCFAVXY      372176
VHLLSXG      353122
CYOAMVC      338057
PWUPWWR      331736
QWAECUS      306916
WDWZKGX      295626
RVMYLQZ      275735
Name: count, dtype: int64

In [4]:
test['F01'].value_counts()

F01
JCDXFYU    2558134
LLKAVMO     872775
VNOHLIR     247349
XOZOJFY     193681
TVWQYKH     108813
VFTOLQT      97665
PWUPWWR      62062
UCFAVXY      61935
VHLLSXG      58178
GLPZFND      57911
QWAECUS      45844
CYOAMVC      44617
WDWZKGX      43936
ZUHRSTZ      43823
RVMYLQZ      41818
Name: count, dtype: int64

In [5]:
# 동일한 결과 보장을 위해 Seed값을 고정합니다
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed를 42로 고정

In [6]:
train = train.groupby('Click').apply(lambda x: x.sample(min(len(x), 5569860)))

In [7]:
train_x = train.drop('Click', axis = 1)
train_y = train['Click']

In [8]:
del train

In [9]:
# Label encode categorical columns
def label_encode_columns(train_df, test_df):
    le = LabelEncoder()
    for column in train_df.columns:
        if train_df[column].dtype == 'object' or isinstance(train_df[column].dtype, pd.CategoricalDtype):
            # Fit the LabelEncoder on the combined data to ensure consistency
            combined_data = pd.concat([train_df[column], test_df[column]], axis=0).astype(str)
            le.fit(combined_data)
            train_df[column] = le.transform(train_df[column].astype(str))
            test_df[column] = le.transform(test_df[column].astype(str))
    return train_df, test_df

# Apply label encoding to train_x and test_x
train_x, test_x = label_encode_columns(train_x, test)

In [10]:
len(train_x)==len(train_y)

True

In [11]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

Optuna 하이퍼파라미터 튜닝

LGBMClassifier(colsample_bytree=0.6829731847990743,
               learning_rate=0.041926565437435385, max_bin=1023,
               min_child_samples=14, n_estimators=6159, n_jobs=-1,
               num_leaves=225, reg_alpha=0.00706357318094864,
               reg_lambda=0.4980866507512539, verbose=-1)

In [12]:
# import optuna
# import lightgbm as lgb
# from lightgbm import early_stopping
# from sklearn import metrics

# def objective(trial):
#     param = {
#         'objective': 'binary',
#         'verbosity': -1,
#         'boosting_type': 'gbdt',
#         'n_jobs':-1,
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
#         'max_bin': 1023,
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 30),
#         'n_estimators': trial.suggest_int('n_estimators', 3000, 8000),
#         'num_leaves': trial.suggest_int('num_leaves', 100, 300),
#         'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 0.1),
#         'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 1.0),
#     }

#     clf = lgb.LGBMClassifier(**param)
#     # Include eval_set and early_stopping_rounds
#     clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[early_stopping(stopping_rounds=10)])
#     preds = clf.predict_proba(X_val)[:, 1]
#     auc = metrics.roc_auc_score(y_val, preds)
#     return auc


In [13]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)


In [14]:
!pip install flaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 9.6 MB/s eta 0:00:00


In [18]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 15.9 MB/s eta 0:00:00


In [15]:
from flaml import AutoML

In [20]:
# Initialize AutoML
automl = AutoML()

automl_settings = {
    "time_budget": 10800,  # Total time budget in seconds
    "metric": 'roc_auc',  # Evaluation metric
    "task": 'classification',  # Task type
    "log_file_name": 'automl.log',  # Log file
    #"estimator_list": ['lgbm', 'xgboost', 'catboost', 'rf', 'extra_tree'],  # List of estimators to use
     "estimator_list": ['lgbm'],
    #"eval_method": "holdout",  # Use holdout validation method
    # "split_ratio": 0.2,  # Ratio of data to be used as validation set
    "early_stop": 10
}


In [ ]:
# Fit the model
automl.fit(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, **automl_settings)

[flaml.automl.logger: 05-29 05:05:00] {1680} INFO - task = classification
[flaml.automl.logger: 05-29 05:05:00] {1688} INFO - Data split method: stratified
[flaml.automl.logger: 05-29 05:05:00] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 05-29 05:05:07] {1789} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 05-29 05:05:07] {1901} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 05-29 05:05:07] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 05-29 05:05:09] {2345} INFO - Estimated sufficient time budget=10891003s. Estimated necessary time budget=10891s.
[flaml.automl.logger: 05-29 05:05:09] {2392} INFO -  at 90.7s,	estimator lgbm's best error=0.3827,	best estimator lgbm's best error=0.3827
[flaml.automl.logger: 05-29 05:05:09] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 05-29 05:05:10] {2392} INFO -  at 91.8s,	estimator lgbm's best error=0.3827,	best estimator lgbm's best error=0.3

In [ ]:
sample_submission = pd.read_csv('drive/MyDrive/data/sample_submission.csv')

# Predict probabilities on the test set
y_pred_test_proba = automl.predict_proba(test_x)[:, 1]

# Output the probabilities for test set
print(y_pred_test_proba)

In [ ]:
sample_submission['Click'] = y_pred_test_proba
sample_submission

In [ ]:
sample_submission.to_csv("drive/MyDrive/data/automl_lgbm_prediction_v2.csv",index = False)